# 2. Dataset Artigos SciELO

## Imports

In [1]:
import os

import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from tqdm.notebook import tqdm

## Código

In [2]:
path_artigos = os.path.join(os.getcwd(), 'artigos_brutos_SciELO')

In [3]:
# contagem de artigos
soma = sum(len(os.listdir(os.path.join(path_artigos, a))) for a in os.listdir(path_artigos))
print(f'Total de artigos: {soma}')

Total de artigos: 1681


### Criando o DataSet

In [4]:
dict_df = {'titulo': [], 'seção': [], 'ano': [],  'doi_id': [], 'texto_html': []}

for ano in tqdm(os.listdir(path_artigos)):
    path_abs = os.path.join(path_artigos, ano)

    for arquivo in tqdm(os.listdir(path_abs), leave=False):
        arq_path = os.path.join(path_abs, arquivo)

        with open(arq_path, 'r', encoding='utf8') as arq:
            texto_html = arq.read()

        soup = BeautifulSoup(texto_html, 'lxml')

        try:
            doi_id = soup.find('a', attrs={'class': '_doi'}).text.strip().removeprefix('https://doi.org/')
        except AttributeError:
            doi_id = None

        try:
            titulo = ' '.join(soup.find('h1', attrs={'class': 'article-title'}).text.split())
            if not titulo:
                titulo = 'SEM TÍTULO'
        except AttributeError:
            titulo = 'SEM TÍTULO'

        try:
            seção = soup.find('span', attrs={'class': '_articleBadge'}).text.strip().lower()
            if not seção:
                seção = 'SEM CATEGORIA'
        except AttributeError:
            seção = 'SEM CATEGORIA'

        dict_df['titulo'].append(titulo)
        dict_df['doi_id'].append(doi_id)
        dict_df['texto_html'].append(texto_html)
        dict_df['seção'].append(seção)
        dict_df['ano'].append(int(ano))


  0%|          | 0/22 [00:00<?, ?it/s]

  0%|          | 0/72 [00:00<?, ?it/s]

  0%|          | 0/71 [00:00<?, ?it/s]

  0%|          | 0/76 [00:00<?, ?it/s]

  0%|          | 0/68 [00:00<?, ?it/s]

...

  0%|          | 0/85 [00:00<?, ?it/s]

  0%|          | 0/109 [00:00<?, ?it/s]

  0%|          | 0/54 [00:00<?, ?it/s]

  0%|          | 0/53 [00:00<?, ?it/s]

  0%|          | 0/107 [00:00<?, ?it/s]

In [5]:
# salvando o dataset em feather
df = pd.DataFrame(dict_df)
df['ano'] = df['ano'].astype(np.uint16)

remover = ['didactic resources', 'articles', 'physics education research']
df = df.query('seção not in @remover')

replace_dict = {
    'cartas ao editor': 'carta ao editor',
    'carta do editor': 'carta ao editor',
    'resenha': 'resenhas',
    'resenhas de livros': 'resenhas',
    'tipo de artigo': 'artigos gerais',
    'artigo gerais': 'artigos gerais'
}

df['seção'] = df['seção'].replace(replace_dict)

In [6]:
replace_dict = {'desenvolvimento em ensino de física': 'pesquisa em ensino de física',
                'história da física': 'história da física e ciências afins',
                'história da física e áreas afins': 'história da física e ciências afins',
                'notas e discussões': 'outros',
                'carta ao editor': 'outros',
                'resenhas': 'outros',
                'editorial': 'outros',
                'errata': 'outros',
                'resenhas de livro': 'outros',
                'notas e discussõoes': 'outros',
                'seção especial: informática no ensino de física': 'seções especiais',
                'seção especial - celebrando os 100 anos de nascimento de richard p. feynman': 'seções especiais',
                'artigos de einstein e ensaios sobre sua obra': 'seções especiais',
                'seção especial: centenário da morte de ludwig boltzmann (1844-1906)': 'seções especiais',
                'seção especial: centenário da morte de william thomson (1824-1907)': 'seções especiais',
                'ano internacional da luz': 'seções especiais',
                'seção especial, história da física e ciências afins': 'seções especiais',
                'artigos de revisão sobre relatividade geral e cosmologia': 'seções especiais',
                'seção especial: encontro " reflexões no ensino de física", sbf/mec ( brasília, 2005)': 'seções especiais',
                'seção especial - ondas gravitacionais': 'seções especiais',
                'seção especial, artigos gerais': 'seções especiais',
                'seção especial - artigos convidados': 'seções especiais', 
                'seção especial, notas e discussões': 'seções especiais',
                'seção especial': 'seções especiais'}

df['nova_seção'] = df['seção'].replace(replace_dict)

In [10]:
df = df.sort_values('ano')
df = df.reset_index(drop=True)
df = df[['titulo', 'ano', 'seção', 'nova_seção', 'doi_id', 'texto_html']]
df.to_feather('scielo_dataset.feather')

In [11]:
pd.read_feather('scielo_dataset.feather').head()

,titulo,ano,seção,nova_seção,doi_id,texto_html
0,Discussão dos Conceitos de Massa Inercial e Ma...,2001,notas e discussões,outros,10.1590/S1806-11172001000300013,"<!DOCTYPE html>\n<div class=""articleTxt"">\n <d..."
1,A influência do fator de vibração térmica na d...,2001,SEM CATEGORIA,SEM CATEGORIA,10.1590/S1806-11172001000200012,"<!DOCTYPE html>\n<div class=""articleTxt"">\n <d..."
2,Perfil de velocidade para o escoamento de flui...,2001,SEM CATEGORIA,SEM CATEGORIA,10.1590/S1806-11172001000200010,"<!DOCTYPE html>\n<div class=""articleTxt"">\n <d..."
3,Uso da Pesquisa em Ensino de Física na prática...,2001,editorial,outros,10.1590/S1806-11172001000300001,"<!DOCTYPE html>\n<div class=""articleTxt"">\n <d..."
4,"Introdução à supercondutividade, suas aplicaçõ...",2001,artigos gerais,artigos gerais,None,"<!DOCTYPE html>\n<div class=""articleTxt"">\n <d..."
